In [63]:
import numpy as np
import math
from scipy.spatial.transform import Rotation as R
cos, sin = np.cos, np.sin

# Problem 1: 
**You are flying in a lunar transit spacecraft and you’re current attitude relative to an inertial frame is given in terms of 3-2-1 Euler angles as (230,70,103). You are interested in docking onto the space station, which attitude relative to the same inertial frame is given through the Euler parameters (β0, β1, β2, β3) = (0.328474, −0.437966, 0.801059, −0.242062). What is the attitude of your spacecraft relative to the space station? Express your answer using the principal rotation angle φ and principal rotation axis eˆ.**

In [89]:
Euler321 = [230, 70, 103]
B0, B1, B2, B3 = [0.328474, -0.437966, 0.801059, -0.242062]

In [90]:
ang1, ang2, ang3 = np.deg2rad(230), np.deg2rad(70), np.deg2rad(103)

Rot1 = np.array([[1, 0, 0], [0, cos(ang3), sin(ang3)], [0, -sin(ang3), cos(ang3)]])
Rot2 = np.array([[cos(ang2), 0, -sin(ang2)], [0,1,0], [sin(ang2), 0, cos(ang2)]])
Rot3 = np.array([[cos(ang1), sin(ang1), 0], [-sin(ang1), cos(ang1), 0], [0,0, 1]])

BN = Rot1 @ Rot2 @ Rot3  #Attitude of spacecraft relative to inertial
BN

array([[-0.21984631, -0.26200263, -0.93969262],
       [-0.76086421, -0.55680095,  0.33325419],
       [-0.61053521,  0.78824319, -0.07693779]])

In [91]:
BN.T.all() == np.linalg.inv(BN).all() #check orthogonality: inverse = transpose

True

In [92]:
SN = np.array([[B0**2 + B1**2 - B2**2 - B3**2, 2*(B1*B2+B0*B3), 2*(B1*B3-B0*B2)],
               [2*(B1*B2-B0*B3), B0**2-B1**2+B2**2-B3**2, 2*(B2*B3+B0*B1)],
               [2*(B1*B3+B0*B2), 2*(B2*B3-B0*B1), B0**2-B1**2-B2**2+B3**2]])

SN #Attitude of station relative to inertial

array([[-0.40058015, -0.86069536, -0.31422426],
       [-0.54265107,  0.49918246, -0.67553278],
       [ 0.73828396, -0.100091  , -0.66702056]])

In [93]:
SN.T.all() == np.linalg.inv(SN).all() #check orthogonality: inverse = transpose

True

In [94]:
NS = SN.T
BS = BN @ NS     #Attitude of spacecraft relative to station
BS 

array([[ 0.60884473,  0.62330588,  0.4907094 ],
       [ 0.67930654, -0.09018562, -0.72829048],
       [-0.40969325,  0.77675876, -0.47832532]])

In [95]:
psi = np.arccos(0.5 * (np.trace(BS) - 1))     #Single angle that takes body orientation to the spacestation
print(psi * 180 / np.pi)

118.67450216657373


In [96]:
e = 1/(2*sin(psi)) * np.array([BS[1,2]-BS[2,1], BS[2,0]-BS[0,2], BS[0,1]-BS[1,0]]) #axis of rotation
e

array([-0.85771497, -0.51313193, -0.03191431])

In [97]:
np.linalg.norm(e) #check rotation axis is normalized

0.9999989327444037

### Results:
The attitude of the spacecraft relative to the space station is expressed by the principle rotation angle $$\Phi = 118.7^o$$ through the principle axis $$ e = \vec(-0.858, -0.513, -0.0319)$$

# Problem 2: 
**The orientation of an object is given in terms of the (3-1-3) Euler angles (−30 , 40 , 20 ), <br>a) find the corresponding principal rotation axis eˆ <br>
b) find the two principal rotation angles Φ and Φ′<br>
c) Find the corresponding Euler parameters.**<br>

In [73]:
ang1, ang2, ang3 = np.deg2rad(-30), np.deg2rad(40), np.deg2rad(20)

In [74]:
Rot1 = np.array([[1, 0, 0], [0, cos(ang2), sin(ang2)], [0, -sin(ang2), cos(ang2)]])
Rot3 = np.array([[cos(ang3), sin(ang3), 0], [-sin(ang3), cos(ang3), 0], [0,0, 1]])
Rot3_ = np.array([[cos(ang1), sin(ang1), 0], [-sin(ang1), cos(ang1), 0], [0,0, 1]])

BN = Rot3 @ Rot1 @ Rot3_
BN

array([[ 0.944799  , -0.24294538,  0.21984631],
       [ 0.06372502,  0.79441526,  0.60402277],
       [-0.3213938 , -0.5566704 ,  0.76604444]])

In [75]:
BN.T.all() == np.linalg.inv(BN).all() #check orthogonality: inverse = transpose

True

 ### Part A:

In [76]:
psi = np.arccos(0.5*(np.trace(BN) - 1))
print(psi * 180 / np.pi)

41.18134335105358


In [77]:
e = 1/(2*sin(psi)) * np.array([BN[1,2]-BN[2,1], BN[2,0]-BN[0,2], BN[0,1]-BN[1,0]]) #axis of rotation
e

array([ 0.88139039, -0.41099909, -0.23287493])

### Part B:

The two principle rotation angles are: $$\Phi = 41.18^o$$ and $$\Phi^{'} = -318.8^o$$

### Part C:

In [87]:
B0 = 0.5 * np.sqrt(np.trace(BN)+1)
B1 = (BN[1,2]-BN[2,1]) / (4*B0)
B2 = (BN[2,0]-BN[0,2]) / (4*B0)
B3 = (BN[0,1]-BN[1,0]) / (4*B0)
print("[B0, B1, B2, B3] = ", [B0, B1, B2, B3])

[B0, B1, B2, B3] =  [0.026429270604954556, -0.8408810072854596, 0.5021588170057916, -0.200142818370456]


# Problem 3:

**Consider the following attitude determination problem. You have 2 attitude sensors (sun sensor and magnetometer). The first measurement Bv1 is the heading to the sun in body frame components. The 2nd measurement Bv2 is the magnetic field heading also in body frame components.**
$$ v_1^B  = \begin{pmatrix} 0.8273 \\ 0.5541 \\ -0.0920 \end{pmatrix} $$ $$v_2^B = \begin{pmatrix} -0.8285\\ 0.5522 \\ -0.0955 \end{pmatrix} $$

**Because you know the satellite’s orbital position, you know that the sun and magnetic field heading vectors are expressed in inertial frame vector components:**

$$v_1^N  = \begin{pmatrix} -0.1517 \\ -0.9669 \\ 0.2050 \end{pmatrix} $$ $$v_2^N = \begin{pmatrix} -0.8393 \\ 0.4494 \\ -0.3044 \end{pmatrix}$$

**a) Use the TRIAD method to determine the spacecraft’s attitude matrix [BN] <br>
b) what are the corresponding (3-2-1) Euler angles?**


In [79]:
v1B = [0.8273, 0.5541, -0.0920] / np.linalg.norm([0.8273, 0.5541, -0.0920])
v2B = [-0.8285, 0.5522, -0.0955] / np.linalg.norm([-0.8285, 0.5522, -0.0955])
v1N = [-0.1517, -0.9669, 0.2050] / np.linalg.norm([-0.1517, -0.9669, 0.2050])
v2N = [-0.8393, 0.4494, -0.3044] / np.linalg.norm([-0.8393, 0.4494, -0.3044])

### Part a:

In [80]:
t1B = v1B
t2B = np.cross(v1B, v2B) / (np.linalg.norm(np.cross(v1B, v2B)))
t3B = np.cross(t1B, t2B)

t1N = v1N
t2N = np.cross(v1N, v2N) / (np.linalg.norm(np.cross(v1N, v2N)))
t3N = np.cross(t1N, t2N)

print(t1B, t2B, t3B)
print(t1N, t2N, t3N)

[ 0.82733471  0.55412325 -0.09200386] [-0.0022758   0.16709806  0.98593766] [ 0.56170464 -0.81549106  0.13950709]
[-0.15170504 -0.96693215  0.20500682] [ 0.21773522 -0.23500418 -0.9472932 ] [ 0.96414571 -0.09907195  0.24618651]


In [81]:
BT = np.array([t1B, t2B, t3B]).T
NT = np.array([t1N, t2N, t3N]).T
print(BT)
print("")
print(NT)

[[ 0.82733471 -0.0022758   0.56170464]
 [ 0.55412325  0.16709806 -0.81549106]
 [-0.09200386  0.98593766  0.13950709]]

[[-0.15170504  0.21773522  0.96414571]
 [-0.96693215 -0.23500418 -0.09907195]
 [ 0.20500682 -0.9472932   0.24618651]]


In [82]:
BT.T.all() == np.linalg.inv(BT).all() #check if matrix is orthogonal

True

In [83]:
TN = NT.T
BN = BT @ TN
BN

array([[ 0.41555875, -0.85509088,  0.31004921],
       [-0.83393237, -0.49427603, -0.24545471],
       [ 0.36313597, -0.15655922, -0.91848869]])

### Part b:

In [84]:
psi = math.atan2(BN[0,1],BN[0,0]) * 180 / np.pi
theta = -np.arcsin(BN[0,2]) * 180 / np.pi
phi = math.atan2(BN[1,2], BN[2,2]) * 180 / np.pi
print('The 321 Euler angles are \u03C8 =', psi, 'and \u03F4 is', theta, 'and \u03C6 is', phi)

The 321 Euler angles are ψ = -64.08108384222732 and ϴ is -18.062195951170207 and φ is -165.03804734205067
